In [ ]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import os
import nltk
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from nltk.corpus import stopwords
import string
input_file = 'Input.xlsx'
df = pd.read_excel(input_file)

In [ ]:
#Read the input file
input_file = 'input.xlsx'
df = pd.read_excel(input_file)


# Create a directory to save the extracted articles
output_dir = 'extracted_articles'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    


# Function to extract article text
def extract_article_text(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # to extract the title and article text
        title = soup.find('title').text
        article_text = ''
        for paragraph in soup.find_all('p'):
            article_text += paragraph.text + '\n'
        return title, article_text
    else:
        return None, None

# Looping through the URLs in the DataFrame URL_ID
for index, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']  
    title, article_text = extract_article_text(url)
    if title and article_text:
        # Generate a unique file name using URL_ID
        file_name = os.path.join(output_dir, f'{url_id}.txt')
        # Write the title and article text to the file
        with open(file_name, 'w', encoding='utf-8') as f:
            f.write(f'Title: {title}\n\n')
            f.write(article_text)
    else:
        print(f'Error extracting: {url} with id :{url_id}')

print('Extraction and saving complete.')

In [ ]:
def get_stop_words(input_file):
    '''
    This function is making a set of stop words found in stop word folder
    '''
    with open('StopWords-20230811T064926Z-001\\StopWords\\'+ input_file,'r') as f:
        content = f.read()
        cleaned_text = re.sub(r'[\n|]', ' ', content)
        return cleaned_text.split()

stop_words =[]
stop_word_dir = os.listdir('StopWords-20230811T064926Z-001\\StopWords\\')
for x in stop_word_dir:
    stop_words = get_stop_words(x)

stop_words = set(stop_words)


In [ ]:
def remove_stop_words(input_file_name):
    '''
    This function is removing stop words from text of extracted file
    '''
    with open('extracted_articles\\'+str(input_file_name) , 'r', encoding='utf-8') as f:
        content = f.read()
        content = content.split()
        content = [word for word in content if word.lower() not in stop_words]
    with open('extracted_articles\\'+str(input_file_name) , 'w', encoding='utf-8') as file:
        content = ' '.join(content)
        file.write(content)


extracted_articles = os.listdir('extracted_articles')
for file_name in extracted_articles:
    remove_stop_words(file_name)

In [ ]:
positive_words = set()
negative_words = set()


def process_file(filename, word_set):
    '''
    This function processes a file and extract words not in stopwords
    '''
    with open(filename, 'r') as file:
        for line in file:
            words = line.split()
            for word in words:
                # Remove punctuation and convert to lowercase
                word = word.strip('.,!?()[]{}"\'').lower()
                # Check if the word is not empty and word not in the stopwords set
                if word and word not in stop_words:
                    word_set.add(word)

                    
positive_file = 'MasterDictionary-20230811T080722Z-001\\MasterDictionary\\positive-words.txt'
negative_file='MasterDictionary-20230811T080722Z-001\\MasterDictionary\\negative-words.txt'


# Processing the positive and negative files
process_file(positive_file, positive_words)
process_file(negative_file, negative_words)

In [ ]:
# Directory where the extracted article files are located
directory = "extracted_articles"

# Initialize empty lists with fixed size
num_files = len(df['URL_ID'])
positive_scores = [0] * num_files
negative_scores = [0] * num_files
polarity_scores = [0] * num_files
subjectivity_scores = [0] * num_files
avg_sentence_lengths = [0] * num_files
percentage_complex_words = [0] * num_files
fog_indices = [0] * num_files
complex_word_counts = [0] * num_files
word_counts = [0] * num_files
syllable_counts_per_word = [0] * num_files
personal_pronoun_counts = [0] * num_files
average_word_lengths = [0] * num_files
average_words_per_sentence = [0] * num_files  
filenames = [0]*num_files
complex_word_counts = [0]*num_files



def calculate_average_sentence_length(text, idx):
    '''
    Function to calculate Average Sentence Length
    '''
    tokens = nltk.word_tokenize(text)
    sentences = nltk.sent_tokenize(text)
    avg_sentence_lengths[idx] = len(tokens) / len(sentences)

    

def calculate_fog_index(idx):
    '''
     Function to calculate Fog Index
    '''
    fog_indices[idx] = 0.4 * (avg_sentence_lengths[idx] + percentage_complex_words[idx])


def calculate_sentiment_scores(tokens, idx):
    '''
    Function to calculate positive and negative scores
    '''
    text_positive_score = sum(1 for token in tokens if token.lower() in positive_words)
    text_negative_score = sum(1 for token in tokens if token.lower() in negative_words)
    
    positive_scores[idx] = text_positive_score
    negative_scores[idx] = text_negative_score
    total_words = len(tokens)
    
    # Calculate polarity and subjectivity scores
    text_polarity_score = (text_positive_score - text_negative_score) / (text_positive_score + text_negative_score + 0.000001)
    text_subjectivity_score = (text_positive_score + text_negative_score) / (total_words + 0.000001)
    polarity_scores[idx] = text_polarity_score
    subjectivity_scores[idx] = text_subjectivity_score

def calculate_personal_pronoun_count(text, idx):
    '''
    Function to calculate Personal Pronoun Count using regex
    '''
    
    personal_pronoun_count = len(re.findall(r'\b(?:I|we|my|ours|us)\b', text, flags=re.IGNORECASE))
    personal_pronoun_counts[idx] = personal_pronoun_count
    

    
def calculate_average_word_length(tokens, idx):
    '''
    Function to calculate Average Word Length
    '''
    cleaned_words = [word for word in tokens if word.lower() not in stop_words and word not in string.punctuation]
    total_characters = sum(len(word) for word in cleaned_words)
    average_word_length = total_characters / len(cleaned_words)
    average_word_lengths[idx] = average_word_length


def calculate_average_words_per_sentence(tokens, idx):
    '''
    Function to calculate Average Words Per Sentence
    '''
    sentences = nltk.sent_tokenize(text)
    average_words_per_sentence[idx] = len(tokens) / len(sentences)
    
def count_syllables(word):
    '''
    Function to count syllables in a word
    '''
    if word.lower():
        syllable_count = max([len(list(y for y in x if y[-1].isdigit())) for x in word])
        
        # Handle exceptions for words ending with "es" or "ed"
        if word.lower().endswith(("es", "ed")):
            syllable_count -= 1
        
        return syllable_count
    else:
        # Default to 1 syllable if word is not found in dictionary
        return 1

def calculate_complex_word_metrics(text, idx):
    '''
    Function to calculate Complex Word Count and Percentage of Complex Words
    '''
    
    
    tokens = nltk.word_tokenize(text)
    complex_word_count = 0  # Initialize the count of complex words
    
    for token in tokens:
        # Remove punctuation from the token
        cleaned_token = ''.join(char for char in token if char not in string.punctuation)
        
        # Check if the cleaned token is a stopword
        if cleaned_token.lower() not in stop_words:
            num_syllables = count_syllables(cleaned_token)
            
            word_counts[idx] += len(cleaned_token)
            syllable_counts_per_word[idx] += num_syllables
            
            # Consider words with more than two syllables as complex
            if num_syllables > 2:
                complex_word_count += 1
                
    
    # Calculate Percentage of Complex Words
    percentage_complex_words[idx] = (complex_word_count / len(tokens)) * 100


# Process each URL_ID in the DataFrame
for idx, url_id in enumerate(df['URL_ID']):
    # Define the file path based on URL_ID
    file_path = os.path.join(directory, f"{url_id}.txt")
    
    try:
        # Check if the file exists
        if os.path.isfile(file_path):
            # Read the content of the file
            with open(file_path, "r", encoding="utf-8") as file:
                text = file.read()
                
                # Tokenize the text
                tokens = nltk.word_tokenize(text)
                
                calculate_average_sentence_length(text, idx)
                calculate_complex_word_metrics(text, idx)
                calculate_fog_index(idx)
                calculate_sentiment_scores(tokens, idx)
                calculate_personal_pronoun_count(text, idx)
                calculate_average_word_length(tokens, idx)
                calculate_average_words_per_sentence(tokens, idx)
                
                # Store complex word count
                complex_word_counts[idx] = calculate_complex_word_metrics(text,idx)
                
                # Store the filename
                filenames[idx] = f"{url_id}.txt"
                
        else:
            # If the file doesn't exist, append NaN values for scores and filename
            filenames[idx] = f"{url_id}.txt"
    except Exception as e:
        print(f"Error processing file '{file_path}': {str(e)}")



        

# Create a new DataFrame with the calculated values
data = {
    'URL_ID': df['URL_ID'],
    'URL': df['URL'],
    'POSITIVE SCORE': positive_scores,
    'NEGATIVE SCORE': negative_scores,
    'POLARITY SCORE': polarity_scores,
    'SUBJECTIVITY SCORE': subjectivity_scores,
    'AVG SENTENCE LENGTH': avg_sentence_lengths,
    'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
    'FOG INDEX': fog_indices,
    'AVG NUMBER OF WORDS PER SENTENCE': average_words_per_sentence,
    'COMPLEX WORD COUNT': complex_word_counts,
    'WORD COUNT': word_counts,
    'SYLLABLE PER WORD': syllable_counts_per_word,
    'PERSONAL PRONOUNS': personal_pronoun_counts,
    'AVG WORD LENGTH': average_word_lengths
}
new_df = pd.DataFrame(data)
new_df.to_excel('output.xlsx' , index = False)

In [1]:
import os
import requests
from bs4 import BeautifulSoup

#Read the input file
url_list = list()
with open('url.txt' , 'r') as f:
    url_list = f.read().split('\n')
    f.close()


# Create a directory to save the extracted articles
output_dir = 'extracted_articles'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    

# Function to extract article text
def extract_article_text(url):
    '''
    This fuction takes url and extract the Title and Paragraph of the Web page and stores
    them into text file.
    '''
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # to extract the title and article text
        title = soup.find('title').text
        article_text = ''
        for paragraph in soup.find_all('p'):
            article_text += paragraph.text + '\n'
        return title, article_text
    else:
        return None, None

# Looping through the URLs in the DataFrame URL_ID
for url in url_list:
    title, article_text = extract_article_text(url)
    if title and article_text:
        # Generate a unique file name using URL_ID
        file_name = os.path.join(output_dir,str(urls[i].split('/')[3]+'.txt'))
        # Write the title and article text to the file
        with open(file_name, 'w', encoding='utf-8') as f:
            f.write(f'Title: {title}\n\n')
            f.write(article_text)
        print
    else:
        print(f'Error extracting: {url} with id :{url_id}')

print('Extraction and saving complete.')

NameError: name 'urls' is not defined